<a href="https://colab.research.google.com/github/madhavibethala/pyspark_examples/blob/main/spark_ex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

sample_data


In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bion

In [3]:
!ls

sample_data  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

In [7]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2021-08-12 04:32:34--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [          <=>       ]   1.62G  3.18MB/s    in 9m 13s  

2021-08-12 04:41:48 (3.00 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1740835817]



In [8]:
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv

In [9]:
data=spark.read.csv('reported_crimes.csv',header=True)
data.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|   

In [10]:
from google.colab import files
uploaded = files.upload()

Saving ToyotaCars.csv to ToyotaCars.csv


In [11]:
data.select('IUCR').show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [12]:
data.select(['IUCR']).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import lit

In [14]:
data.withColumn("new_col",lit(1)).show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|new_col|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY .

In [17]:
data=data.drop('new_col')
data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE|

***Working with Rows***

In [20]:
from pyspark.sql.functions  import to_timestamp,colfilter(col('Date') == lit('2018-11-12'))
rc.count()
rc = spark.read.csv('reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).

3

In [21]:
rc.show(5)

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false| 

In [22]:
 data.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  51805|
|CRIMINAL SEXUAL A...|   3545|
|            STALKING|   4041|
|PUBLIC PEACE VIOL...|  51122|
|           OBSCENITY|    739|
|NON-CRIMINAL (SUB...|      9|
|               ARSON|  12467|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14602|
|   CRIMINAL TRESPASS| 207113|
|             ASSAULT| 471427|
|      NON - CRIMINAL|     38|
|LIQUOR LAW VIOLATION|  14531|
| MOTOR VEHICLE THEFT| 339930|
|               THEFT|1552159|
|             BATTERY|1352535|
|             ROBBERY| 276823|
|            HOMICIDE|  11296|
|           RITUALISM|     24|
|    PUBLIC INDECENCY|    187|
+--------------------+-------+
only showing top 20 rows



In [30]:
data.groupBy('Primary Type').count().orderBy('count',ascending=False).show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|               THEFT|1552159|
|             BATTERY|1352535|
|     CRIMINAL DAMAGE| 840847|
|           NARCOTICS| 738680|
|             ASSAULT| 471427|
|       OTHER OFFENSE| 458721|
|            BURGLARY| 411199|
| MOTOR VEHICLE THEFT| 339930|
|  DECEPTIVE PRACTICE| 315667|
|             ROBBERY| 276823|
|   CRIMINAL TRESPASS| 207113|
|   WEAPONS VIOLATION|  91541|
|        PROSTITUTION|  69422|
|OFFENSE INVOLVING...|  51805|
|PUBLIC PEACE VIOL...|  51122|
|         SEX OFFENSE|  28419|
| CRIM SEXUAL ASSAULT|  27865|
|INTERFERENCE WITH...|  17685|
|            GAMBLING|  14602|
|LIQUOR LAW VIOLATION|  14531|
+--------------------+-------+
only showing top 20 rows



find the percentage of reported crimes that results in arrest

In [31]:
data.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [44]:
arst_count=data.filter(col('Arrest') == 'true').count()
arst_count
#arest_data.show()
tot_count=data.select(col('Arrest')).count()
tot_count


7376983

In [65]:
data.filter(col('Arrest') == 'true').count()/data.select('Arrest').count()

0.269245977657804

In [45]:
print(arst_count)
print(tot_count)

1986223
7376983


In [40]:
data.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [43]:
per_arr=arst_count/tot_count*100
per_arr

26.9245977657804

**find top 3 locations on reported crime**

In [47]:
data.select('Primary Type').distinct().show()

+--------------------+
|        Primary Type|
+--------------------+
|OFFENSE INVOLVING...|
|CRIMINAL SEXUAL A...|
|            STALKING|
|PUBLIC PEACE VIOL...|
|           OBSCENITY|
|NON-CRIMINAL (SUB...|
|               ARSON|
|   DOMESTIC VIOLENCE|
|            GAMBLING|
|   CRIMINAL TRESPASS|
|             ASSAULT|
|      NON - CRIMINAL|
|LIQUOR LAW VIOLATION|
| MOTOR VEHICLE THEFT|
|               THEFT|
|             BATTERY|
|             ROBBERY|
|            HOMICIDE|
|           RITUALISM|
|    PUBLIC INDECENCY|
+--------------------+
only showing top 20 rows



In [62]:
temp_data=data.filter(col('Location') !='')
#arst_count=data.filter(col('Arrest') == 'true').count()
temp_data.show()

+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|   Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|        BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    t

In [88]:
#data.select('Location').groupBy().count()


data.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1914539|
|           RESIDENCE|1252450|
|           APARTMENT| 800008|
+--------------------+-------+
only showing top 3 rows



In [78]:
d1 = {'name': ['ravi','savi'], 'age': [45,78] ,'marks': [67,89]}
print(d1.items())
l1 = list(d1.items())
print(l1)
print(d1.keys())
print(d1.values())
print(list(d1.values()))

dict_items([('name', ['ravi', 'savi']), ('age', [45, 78]), ('marks', [67, 89])])
[('name', ['ravi', 'savi']), ('age', [45, 78]), ('marks', [67, 89])]
dict_keys(['name', 'age', 'marks'])
dict_values([['ravi', 'savi'], [45, 78], [67, 89]])
[['ravi', 'savi'], [45, 78], [67, 89]]


In [81]:
temp=[]
dictList = []

for key, value in d1.items():
    aKey = key
    aValue = value
    temp.append(aKey)
    temp.append(aValue)
    dictList.append(temp) 

print(dictList)   
x=dict(dictList)
print(x) 

[['name', ['ravi', 'savi'], 'age', [45, 78], 'marks', [67, 89]], ['name', ['ravi', 'savi'], 'age', [45, 78], 'marks', [67, 89]], ['name', ['ravi', 'savi'], 'age', [45, 78], 'marks', [67, 89]]]


ValueError: ignored

In [86]:
sam_dic={'a':1,'':2,'':3}
print(sam_dic.keys())

dict_keys(['a', ''])


In [89]:
data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE|

**display the primery type col in lower and upper characters ,first 4 characters of the primery type**

In [90]:
from pyspark.sql.functions import lower,upper,substring

In [98]:
data.select(lower(col('Primary Type')),upper(col('Primary Type')),substring(col('Primary Type'),1,4)).show(5)

+-------------------+-------------------+-----------------------------+
|lower(Primary Type)|upper(Primary Type)|substring(Primary Type, 1, 4)|
+-------------------+-------------------+-----------------------------+
|            battery|            BATTERY|                         BATT|
|              theft|              THEFT|                         THEF|
|              theft|              THEFT|                         THEF|
|          narcotics|          NARCOTICS|                         NARC|
|            assault|            ASSAULT|                         ASSA|
+-------------------+-------------------+-----------------------------+
only showing top 5 rows



In [101]:
from pyspark.sql.functions import min,max

data.select(min(col('Date')),max(col('Date'))).show()

+--------------------+--------------------+
|           min(Date)|           max(Date)|
+--------------------+--------------------+
|01/01/2001 01:00:...|12/31/2020 12:55:...|
+--------------------+--------------------+



In [102]:
from pyspark.sql.functions import date_add,date_sub
help(date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]
    
    .. versionadded:: 1.5



In [110]:
data.select(date_add(min(col('Date')),3)).show(1)
#data.select(date_sub(min(col('Date')),2)).show(1)

+----------------------+
|date_add(min(Date), 3)|
+----------------------+
|                  null|
+----------------------+

